In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys
%matplotlib inline

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [4]:
data = pd.read_csv('./data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                     'product_id': 'item_id'},
            inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [7]:
item_features = pd.read_csv('./data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [9]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns = ['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [14]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [18]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id',
                                  columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0)

user_item_matrix = user_item_matrix.astype(float)

sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## ALS

In [22]:
def get_recommendations(user, model, N=5):
    """Get recommendations."""
    
    res = [id_to_itemid[rec[0]] for rec in model.recommend(userid=userid_to_id[user], 
                                                           user_items=sparse_user_item,
                                                           N=N, 
                                                           filter_already_liked_items=False, 
                                                           filter_items=None, 
                                                           recalculate_user=True)]
    
    return res

In [24]:
%%time

factors = [32, 64, 128]
regularizations = [0.001, 0.01, 0.05, 0.1]
iterations = [1, 5, 10, 15, 50, 100]

for factor in factors:
    for regularization in regularizations:
        for iteration in iterations:
            model = AlternatingLeastSquares(factors=factor,
                                            regularization=regularization,
                                            iterations=iteration,
                                            calculate_training_loss=True,
                                            num_threads=10,
                                            use_gpu=False)
            
            model.fit(csr_matrix(user_item_matrix).T.tocsr(),
                      show_progress=True)
            
            params = str(factor) + '_' + str(regularization) + '_' + str(iteration)
            
            result[params] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
            
#             result.apply(lambda row: precision_at_k(row[params], row['actual']), axis=1).mean()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Wall time: 27min 14s


In [25]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.1831:32 0.001 1
0.1595:32 0.001 5
0.1605:32 0.001 10
0.1601:32 0.001 15
0.1623:32 0.001 50
0.1618:32 0.001 100
0.193:32 0.01 1
0.1677:32 0.01 5
0.1627:32 0.01 10
0.164:32 0.01 15
0.1592:32 0.01 50
0.1595:32 0.01 100
0.198:32 0.05 1
0.1663:32 0.05 5
0.1616:32 0.05 10
0.1647:32 0.05 15
0.1618:32 0.05 50
0.161:32 0.05 100
0.1947:32 0.1 1
0.1617:32 0.1 5
0.1642:32 0.1 10
0.1627:32 0.1 15
0.16:32 0.1 50
0.1611:32 0.1 100
0.1867:64 0.001 1
0.1593:64 0.001 5
0.1556:64 0.001 10
0.153:64 0.001 15
0.1524:64 0.001 50
0.15:64 0.001 100
0.199:64 0.01 1
0.1619:64 0.01 5
0.157:64 0.01 10
0.1554:64 0.01 15
0.1497:64 0.01 50
0.1501:64 0.01 100
0.2065:64 0.05 1
0.1645:64 0.05 5
0.1601:64 0.05 10
0.1556:64 0.05 15
0.1507:64 0.05 50
0.1516:64 0.05 100
0.2131:64 0.1 1
0.162:64 0.1 5
0.1606:64 0.1 10
0.1544:64 0.1 15
0.155:64 0.1 50
0.151:64 0.1 100
0.1972:128 0.001 1
0.1575:128 0.001 5
0.147:128 0.001 10
0.1428:128 0.001 15
0.1392:128 0.001 50
0.1411:128 0.001 100
0.2001:128 0.01 1
0.1644:128 

In [34]:
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id',
                                  columns='item_id',
                                  values='sales_value',
                                  aggfunc='sum',
                                  fill_value=0)

user_item_matrix = user_item_matrix.astype(float)

sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.24,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.24,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
5,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2498,41.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.00,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
%%time

model = AlternatingLeastSquares(factors=64,
                                regularization=0.05,
                                iterations=15,
                                calculate_training_loss=True,
                                num_threads=10,
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),
          show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 1.16 s


In [31]:
%%time

result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

KeyError: 'als'

In [33]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.0971371170266192